# Progetto data Analytics

## Installing libraries

In [1]:
#for install libraries
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
libxml2 set to manually installed.
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
     |████████████████████████████████| 71kB 2.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built cairocffi
     |████████████████████████████████| 378kB 2.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eeba

#### Execute this only if you are Alex Caselli on Colab

In [2]:
from google.colab import drive
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
from tqdm import tqdm_notebook as tqdm
drive.mount('/content/gdrive')
baseURL = '/content/gdrive/My Drive/Data Analytics/';

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Import Libraries and set local path

In [0]:
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
import csv
from tqdm import tqdm_notebook as tqdm

baseURL = '../'

# Creation of a dictionaty for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, allGraph, compactallGraph)

In [3]:

# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph(directed=True)
messageGraph = ig.Graph(directed=True)
tradeGraph = ig.Graph(directed=True)
totalGraph = ig.Graph(directed=True)
allianceGraph = ig.Graph(directed=True)
compactAllGraph = ig.Graph(directed=True)

for i in tqdm(range(0, nGraph)):
    # print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph, compactAllGraph) 
    
# tricksterAnalysis(allNetwork)

# for i in tqdm(range(0, len(allNetwork))):
#    for j in tqdm(range(0, len(allNetwork[i]))):
        # print(allNetwork[i][4].summary())
#        addCommunitiesLabel(allNetwork[i][j], i)
        # print(allNetwork[i][4].summary())
# print("")

# This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [0]:
print(len(allNetwork[0][0].vs))
# 3861


cont = 0
communityContIndegree = 0
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][4].vs:
        if vertex['id'] == '2':
            print("{:>20}{:>3}".format("PRESENT IN THE DAY:", str(i + 1)))
            print("{:>20}{:>3}".format("DAY INDEGREE: ", vertex.indegree()))
            print("{:>20}{:>3}".format("DAY OUTDEGREE: ", vertex.outdegree()))     
            print("{:>20}{:>3}".format("COMMUNITY: ", vertex['Community'])) 
            for edge in allNetwork[i][4].es:
                if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2':
                    if edge['Label'] == 'Attack' and allNetwork[i][4].vs[edge.tuple[1]]['id'] == '4878':
                        print("DIOCANE: ", edge['Weight'])
                    if edge['Label'] == 'Trade':
                        cont += edge['Weight']
                    print("{:>20}{:>3}".format("OUTDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
                if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2':
                    if allNetwork[i][4].vs[edge.tuple[0]]['Community'] == allNetwork[i][4].vs[edge.tuple[1]]['Community'] and i == 26:
                        communityContIndegree += 1
                    print("{:>20}{:>3}".format("INDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
            print("") 
print(cont)
print("COMMUNITY CONT INDEGREE: ", communityContIndegree)

# vertex = allNetwork[1][4].vs.select(id = "10935")
# print(vertex['id'])
# print(vertex['Community'])
    
'''for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
            print(edge)
print("")
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
            print(edge)'''


# 11850

### Domande da fare
**1)** Il numero massimo e minimo di nodi, dato il paper, è corretto e ci torna; il numero minimo e massimo di archi invece no. Il fatto è molto strano perché noi semplicemente apriamo il file graphml e stampiamo le info associate. Lavoriamo facendo finta di nulla? <br>
1) è stato detto di fregarsene, l'importante è che si usi o il file gml oppure il csv, non bisogna usare entrambi

**2)** Abbiamo osservato come, dato il file di community e la rete totale (attack + trade + message), non tutti i nodi sono presenti in questo file. Indagando su questi abbiamo poi osservato che subiscono (nella maggior parte dei casi) attacchi. Possiamo quindi dedurre che si tratti effettivamente di nodi non affiliati a nessuna alleanza in quanto utenti che hanno smesso di giocare o stanno iniziando a giocare (alcuni non subiscono attacchi ma commerciano e attaccano, scudo prottetivo 30 giorni, punteggio minimo giocare)? <br>
2) è normale in quanto le community segnate sono quelle registrate dagli utenti stessi sul server, quindi è possibile che alcuni nodi non siano affiliati a nessuna alleanza

**3)** Le misure di centralità sono possibili per reti multigrafo? <br>
**4)** In cosa differisce la presentazione dalla demo? <br>
**5)** Le community di dimensione minore di 10 come possiamo tratterle in maniera intelligente? Le associamo tutte a uno stesso identificativo, le eliminiamo, che facciamo?


# START ANALYSIS !!!!

### Users number trend in the 30 days

In [0]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", round(sum(nUsers)/len(nUsers)), 2))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")
    return nUsers


def userCommunityNumberTrend(communityStory):
    nUsers = [0]*len(communityStory)
    with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/nUserTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'quantity'])
        
        for i in range(0, len(communityStory)):
            nUsers[i] = communityStory[i].vcount()
            filewriter.writerow([i+1, communityStory[i].vcount()])
        


    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", round(sum(nUsers)/len(nUsers)), 2))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
        
    print("")
    #return nUsers

### Edges number trend in the 30 days

In [0]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += edge['Weight']
            elif edge['Label'] == "Trade":
                nTrade[i] += edge['Weight']
            elif edge['Label'] == "Message":
                nMessage[i] += edge['Weight']
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")
    
    return nAttack, nTrade, nMessage


def edgeCommunityNumberTrend(communityStory):
    nTrade = [0]*len(communityStory)
    nMessage = [0]*len(communityStory)
    
    with open(baseURL+'Dash/SingleCommunityStudy/CSVFILE/nEdgeTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        for i in range(0, len(communityStory)):
            for edge in communityStory[i].es:
                nTrade[i] += edge['TWeight']
                nMessage[i] += edge['MWeight']
            filewriter.writerow([i+1, 'nMessage', nMessage[i]])
            filewriter.writerow([i+1, 'nTrade', nTrade[i]])
                
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")  

Code for generate a csv with edge and nodes trends in 30days

In [0]:
nAttack, nTrade, nMessage=edgeNumberTrend(allNetwork)
nUsers=userNumberTrend(allNetwork)


#write trend in a CSV
with open(baseURL+'Dash/userAndEdge_trends_30days.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['day', 'type', 'quantity'])
    
    for i in range(1, 30):
        filewriter.writerow([i, 'nUsers',nUsers[i]])
        filewriter.writerow([i, 'nAttack',nAttack[i]])
        filewriter.writerow([i, 'nTrade',nTrade[i]])
        filewriter.writerow([i, 'nMessage',nMessage[i]])

Number of attacks in the 30 days: [32997.0, 32782.0, 30264.0, 23883.0, 18748.0, 22424.0, 23448.0, 22550.0, 22401.0, 22778.0, 22162.0, 21628.0, 20675.0, 22086.0, 19969.0, 22134.0, 22367.0, 21772.0, 18820.0, 18390.0, 18771.0, 16970.0, 16477.0, 16467.0, 15715.0, 18164.0, 17230.0, 17057.0, 16513.0, 16749.0]

Number of trades in the 30 days: [9390.0, 8710.0, 8667.0, 8545.0, 9499.0, 9799.0, 9638.0, 8806.0, 8698.0, 8921.0, 9008.0, 9310.0, 9298.0, 9834.0, 10175.0, 9866.0, 9884.0, 9008.0, 8624.0, 8314.0, 8443.0, 8981.0, 8062.0, 8659.0, 8054.0, 8436.0, 9062.0, 8665.0, 8606.0, 8521.0]

Number of messages in the 30 days: [15831.0, 14600.0, 14847.0, 12765.0, 14852.0, 14141.0, 13978.0, 14166.0, 15675.0, 14122.0, 12030.0, 14045.0, 14207.0, 12767.0, 11405.0, 13180.0, 11979.0, 12482.0, 14510.0, 10314.0, 11110.0, 13327.0, 12889.0, 11486.0, 7996.0, 7697.0, 9071.0, 8631.0, 7388.0, 8214.0]

   Average number of attacks in 30 days:  21079.7
  Average number of trades a in 30 days:   8982.8
Average number of

### Trickster detection tool 
This tool is designed for identifying dishonest users who manage two accounts and use one for the main account. The assumption is: if i see you two times do more trading than any else action you are a suspected and is necessary to see what are you done in the period (30 days). 

In [0]:
# Main function call all function that compose the tool
def tricksterAnalysis(allNetwork):
    
    # make a list with main suspect for every day, in the end we have a following structure
    #{DAY: {idSuspected: {idBeneficiry: qta}}} 
    # a dictionary where for first external key we have a day, inside 1 lvl key is suspected id, in the last 2 lvl
    # a dictionary of beneficiary key with quantity of trading
    dailySuspicions = dict()
    print("Start trickester Analysis")
    print("Watching for dishonest user in the period...")
    for i in tqdm(range(0, len(allNetwork))):
        # print("DAY: ", str(i+1))
        dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])
    
    # from first skimming compute a guilty index and print the player with more then 0.3 on index
    edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

# create a day dictionary structur (see below)
def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    # get, specific net, all vertex with 0 indegree
    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    # print(zeroIndegreeVertex)
    
    # create a dictionary for keeping suspects with respective beneficiary
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    # for every zero indegree vertex count the number of edge and the number of trade edge
    # if find a trade edge save the beneficiary of this trade and quantity
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += edge['Weight']
                if edge['Label'] == 'Trade':
                #    if totalNet.vs[edge.tuple[0]]['id'] == vertex['id'] == "7855":
                #        print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += edge['Weight']
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = edge['Weight']
                    nTrades += edge['Weight']
        # if the trades edges number is more than 95%, the node is suspected, saved it!
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    # for key in suspecteds:
    #    print(key, suspecteds[key])
    return suspecteds    

# from all day suspects we want a more compact structure for see what happen to our suspects node in the period
# (30 days). The structure is a dictionary: 
# {idSuspected: [nWatchTimes, {idBeneficiary: [nWatchTimes, qnt]}, nWatchSuspectAsBeneficiary, qnt]}
# from this we can delete from suspects list the node with only one appearance
def printRealSuspected(allSuspects, nDays):
    
    print("Compact recovered information...")
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            # increase the number of nWatchTimes about suspects
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            # for all suspect beneficiary increase nWatchTimes about beneficiary and qnt about trade
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    # increase nWatchSuspectAsBeneficiary qnt index
    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    # order from the most observed to the least observed 
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    # print("Complete: ", len(suspectsSummary))
    # for suspect in suspectsSummary:
    #    print(suspect, suspectsSummary[suspect])
    
    # if suspect is observed only one time or is observed as many times as there are beneficiaries; 
    # is no longer a suspect
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
 
# compute a Guiltyindex (based to experimental observations) and print the node with more than .3
def edgesCount(finalSuspects, allNet):
    print("This are the main suspects!")
    edgesInCount = dict()
    edgesOutCount = dict()
    tradeEdgesInCount = dict()
    edgesOutPlayer = dict()
    finalResult = dict()
 
    activityDay = list()
    attackDay = list()
    tradeDay = list()
    
    inCount = 0
    outCount = 0
    tradeOutCount = 0
    tradeInCount = 0
    guiltyIndex = 0
    maxIndex = 0
    cont = 0
    
    
    # count number of edge about all typology
    for suspect in finalSuspects:
        
        activityDay.clear()
        attackDay.clear()
        tradeDay.clear()
        
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        tradeEdgesInCount.clear()
        
        
        for i in range(0, 3):
            edgesInCount['Attack'] = 0
            edgesInCount['Trade'] = 0
            edgesInCount['Message'] = 0
            edgesOutCount['Attack'] = 0
            edgesOutCount['Trade'] = 0
            edgesOutCount['Message'] = 0
        
        inCount = 0
        outCount = 0
        tradeOutCount = 0
        tradeInCount = 0
        guiltyIndex = 0
        
        # append a activity day of suspect
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))
            

            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += edge['Weight']
                    else:
                        edgesOutCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += edge['Weight']
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = edge['Weight']
                        if (day + 1) not in tradeDay:
                            tradeDay.append(day + 1)
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] == 'Trade':
                        if allNet[day][4].vs[edge.tuple[0]]['id'] in tradeEdgesInCount:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] += edge['Weight']
                        else:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] = edge['Weight']
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += edge['Weight']
                    else:
                        edgesInCount[edge['Label']] = edge['Weight']
                    if edge['Label'] == 'Attack':
                        if (day + 1) not in attackDay:
                            attackDay.append(day + 1)
        guiltyIndex = edgesOutCount['Trade']*((1/len(edgesOutPlayer)))*1.3
        # print("Trade out: ", edgesOutCount['Trade'])
        # print("People Served: ", len(edgesOutPlayer))
        # print(guiltyIndex)
        guiltyIndex -= (edgesInCount['Trade']*5)
        # print(guiltyIndex)
        guiltyIndex -= (edgesOutCount['Message'])*1
        # print(guiltyIndex)
        guiltyIndex += (edgesInCount['Message'])*0.3
        # print(guiltyIndex)
        guiltyIndex -= edgesOutCount['Attack']*0.4
        # print(guiltyIndex)
        guiltyIndex -= edgesInCount['Attack']*0.08
        # print(guiltyIndex)
        # print("DENOMINATOR: ", edgesInCount['Attack']+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']+edgesOutCount['Trade'])
        guiltyIndex = guiltyIndex / (edgesInCount['Attack']*0.08+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']*2+edgesOutCount['Trade'])
        if guiltyIndex >= 0.3:
            if guiltyIndex > maxIndex:
                maxIndex = guiltyIndex
            print("GULTY INDEX: ", round(guiltyIndex, 4))
            print("SUSPECT: ", suspect)
            print("EDGES IN: ", edgesInCount)
            print("EDGES OUT: ", edgesOutCount)
            print("PLAYER SERVED: ", edgesOutPlayer)
            print("ACTIVITY DAYS: ", activityDay)
            print("DAYS OF ATTACKS SUFFERED: ", attackDay)
            print("DAYS OF TRADE DONE: ", tradeDay)
            print("PLAYER TRADE WITH SUSPECT: ", tradeEdgesInCount)
            print(" ")
            
            value = [0]*8
            finalResult[suspect] = list([0, dict(), dict(), dict(), dict(), list(), list(), dict()])
            finalResult[suspect][0] = round(round(guiltyIndex, 4), 3)
            finalResult[suspect][1] = edgesInCount
            finalResult[suspect][2] = edgesInCount
            finalResult[suspect][3] = edgesOutCount
            finalResult[suspect][4] = edgesOutPlayer
            finalResult[suspect][5] = activityDay
            finalResult[suspect][6] = attackDay
            finalResult[suspect][7] = tradeEdgesInCount
            cont += 1
            
    for suspect in finalResult:
        print(suspect)
        finalResult[suspect][0] = finalResult[suspect][0]/maxIndex
        print(finalResult[suspect])
    
    
    print(cont)

## Communities Dynamics

#### Add communities label to vertex

In [0]:
def addCommunitiesLabel(allCompactNetwork, dayCommunity):
        
    # print("DAY: ", (dayCommunity+1))
    communities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(dayCommunity + 1)+".txt", "r")
    
    communityCont = 0
    nNodes = 0
    nNodesPresent = 0
    nDissolved = 0
    nActive = 0
    
    present = False
    
    communityAffiliates = dict()
    
    affiliatesNodes = list()
    noMatchNodes = list()
    nodesGraphNoMatch = list()
    vertexToDelete = list()
    
    for line in communities:
        # print("COMMUNITY NUMBER: ", cont)
        # print(line)
        line = line.split()
        # print(type(line[0]))
        for node in line:
            if node == "DISSOLVED":
                nDissolved += 1
                affiliatesNodes.append("DISSOLVED")
                continue
            present = False
            nNodes += 1
            # allCompactNetwork.vs.select('id' = node)['Community'] = communityCont
            for vertex in allCompactNetwork.vs:
                if vertex['id'] == str(node):
                    if len(line) > 10:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = communityCont
                      #  print("VERTEX IS: ", vertex)
                        if vertex.outdegree() > 0:
                            nActive += 1
                    else:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = -communityCont
                        if vertex.outdegree() > 0:
                            nActive += 1
                    nNodesPresent += 1
                    present = True
            if not present:
                noMatchNodes.append(node)
       
        communityAffiliates[communityCont] = list((list(affiliatesNodes), len(line), nActive))
        
        
        
        communityCont += 1
        affiliatesNodes.clear()
        nActive = 0
        
    # print("NUMBER OF NODES IN DAY TOTAL GRAPH: ", len(allCompactNetwork.vs))
    
    for vertex in allCompactNetwork.vs:
        if vertex['Community'] is None:
            nodesGraphNoMatch.append(vertex['id'])
            vertexToDelete.append(vertex.index)
            # print(vertex.index)
            # print(vertex)
    allCompactNetwork.delete_vertices(vertexToDelete)
        
    # print("NUMBER OF NODES IN COMMUNITIES FILE: ", nNodes)
    # print("NUMBER OF COMMUNITIES: ", communityCont)
    # print("NUMBER OF DISSOLVED COMMUNITIES: ", nDissolved)
    # print("NUMBER OF REAL COMMUNITIES: ", (communityCont-nDissolved))
    # print("NODES IN COMMUNITIES DAY FILE NO PRESENT IN DAY GRAPH: ", noMatchNodes)
    # print("NUMBER OF NODES IN COMMUNITIES DAY FILE NL PRESENT IN DAY GRAPH: ", len(noMatchNodes))
    # print("NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", nodesGraphNoMatch)
    # print("NUMBER OF NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", len(nodesGraphNoMatch))
    # print("NUMBER OF NODES IN COMMUNITIES DAY FILE PRESENT IN DAY GRAPH: ", nNodesPresent)
    # print("ALL DAY GRAPH AFTER REMOVING THE NO COMMUNITIES NODE: ", allCompactNetwork.summary())
    # print("----------------------------------")
    return communityAffiliates


def communityMemberTrend(communityAffiliates, communityId):
    
    daySituation = [0]*len(communityAffiliates)
    
    
    for day in communityAffiliates:
        if communityId in communityAffiliates[day]:
            if "DISSOLVED" not in communityAffiliates[day][communityId][0]:
                daySituation[day] = (len(communityAffiliates[day][communityId][0]), communityAffiliates[day][communityId][1])
            else: 
                daySituation[day] = ("DISSOLVED", 0)
        else:
            daySituation[day] = ("NOT PRESENT", 0)
    for i in range(0, len(daySituation)):
        print("{:<36}{:<3}".format("DAY: ", (i+1)))
        print("{:<36}{:<3}".format("NUMBER OF MEMBER ACTIVE IN THE NET: ", daySituation[i][0]))
        print("{:<36}{:<3}".format("TOTAL NUMBER IN COMMUNITY: ", daySituation[i][1]))
        print("")
        
def communitiesNumberTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesNumberTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
    
        for day in communityAffiliates:
            for community in communityAffiliates[day]:
                if "DISSOLVED" not in communityAffiliates[day][community][0]:
                    daySituation[day] += 1
    
        for i in range(0, len(daySituation)):
            print("{:<30}{:<3}".format("DAY: ", (i+1)))
            print("{:<30}{:<3}".format("NUMBER OF COMMUNITY IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfCommunities', daySituation[i]])
        
            print("")           
    
def dissolvedCommunitiesTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesDissolvedTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
        nDissolved = 0
    
        for day in communityAffiliates:
            if day == 0:
                daySituation[day] = 0
                continue
            for community in communityAffiliates[day]:
                if "DISSOLVED" in communityAffiliates[day][community][0]:
                    daySituation[day] += 1
            if daySituation[day] == daySituation[day - 1]:
                daySituation[day] = 0
            else:
                daySituation[day] = daySituation[day] - nDissolved
                nDissolved += daySituation[day]
    
        for i in range(0, len(daySituation)):
            print("{:<42}{:<3}".format("DAY: ", (i+1)))
            print("{:<42}{:<3}".format("NUMBER OF DISSOLVED COMMUNITY IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfDissolved', daySituation[i]])
            print("") 
        
    print(nDissolved)
        
def creationCommunitiesTrend(communityAffiliates):
    
    with open(baseURL+'Dash/communitiesCreationTrend.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['day', 'type', 'quantity'])
    
        daySituation = [0]*len(communityAffiliates)
    
        for day in communityAffiliates:
            if day == 0:
                daySituation[0] = 0
                continue
            if len(communityAffiliates[day]) - len(communityAffiliates[day - 1]) > 0:
                daySituation[day] = len(communityAffiliates[day]) - len(communityAffiliates[day - 1])
            else:
                daySituation[day] = 0
        
        for i in range(0, len(daySituation)):
            print("{:<42}{:<3}".format("DAY: ", (i+1)))
            print("{:<42}{:<3}".format("NUMBER OF COMMUNITIES CREATED IN THE NET: ", daySituation[i]))
            filewriter.writerow([i + 1, 'NumberOfCreated', daySituation[i]])
            print("") 
    
def retrieveCommunityGraph(allNetwork, communityId, netId):
    
    communityDayGraph = dict()
    dayGraph = ig.Graph(directed=True)
    nVertex = 0
    
    for day in range(0, 30):
        nVertex = 0
        dayGraph = ig.Graph(directed=True)
        for vertex in allNetwork[day][netId].vs:
            if abs(vertex['Community']) == communityId:
                dayGraph.add_vertex(vertex['id'])
                dayGraph.vs[nVertex]['Community'] = vertex['Community'] 
                dayGraph.vs[nVertex]['label'] = vertex['id'] 
                nVertex += 1
                
        for edge in allNetwork[day][netId].es:
            for vertex in dayGraph.vs:
                if allNetwork[day][netId].vs[edge.tuple[0]]['id'] == vertex['name'] and abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) == communityId:
                    toVertex = allNetwork[day][netId].vs[edge.tuple[1]]['id']
                    if  'Label' in edge.attribute_names():
                        if edge['Label'] == 'Attack':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "red")
                        elif edge['Label'] == 'Message':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "blue")
                        else:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "orange")

                    elif 'MWeight' in edge.attribute_names() and 'TWeight' in edge.attribute_names():
                        if edge['MWeight'] > 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "darkgreen")
                        elif edge['MWeight'] > 0 and edge['TWeight'] == 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "blue")
                        elif edge['MWeight'] == 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "orange")
                    else:
                        dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, Weight = edge['Weight'])

                        
        communityDayGraph[day] = dayGraph
    
    return communityDayGraph     

def retrieveCommunitiesGraph(allNetwork, communityListId, netId):
    
    communityDayGraph = dict()
    dayGraph = ig.Graph(directed=True)
    nVertex = 0
    
    for day in range(0, 30):
        nVertex = 0
        dayGraph = ig.Graph(directed=True)
        for vertex in allNetwork[day][netId].vs:
            if abs(vertex['Community']) in communityListId:
                dayGraph.add_vertex(vertex['id'])
                dayGraph.vs[nVertex]['Community'] = vertex['Community'] 
                dayGraph.vs[nVertex]['label'] = vertex['id'] 
                nVertex += 1
                
        for edge in allNetwork[day][netId].es:
            for vertex in dayGraph.vs:
                if allNetwork[day][netId].vs[edge.tuple[0]]['id'] == vertex['name'] and (abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) in communityListId or abs(allNetwork[day][netId].vs[edge.tuple[1]]['Community']) in communityListId):
                    toVertex = allNetwork[day][netId].vs[edge.tuple[1]]['id']
                    if  'Label' in edge.attribute_names():
                        if edge['Label'] == 'Attack':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "red")
                        elif edge['Label'] == 'Message':
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "blue")
                        else:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, key = edge['key'], Weight = edge['Weight'], Label = edge['Label'], color = "orange")

                    elif 'MWeight' in edge.attribute_names() and 'TWeight' in edge.attribute_names():
                        if edge['MWeight'] > 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "darkgreen")
                        elif edge['MWeight'] > 0 and edge['TWeight'] == 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "blue")
                        elif edge['MWeight'] == 0 and edge['TWeight'] > 0:
                            dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, MWeight = edge['MWeight'], TWeight = edge['TWeight'], color = "orange")
                    else:
                        dayGraph.add_edge(vertex.index, dayGraph.vs.select(name = toVertex)[0].index, Weight = edge['Weight'])

                        
        communityDayGraph[day] = dayGraph
    
    return communityDayGraph   

#### Communities War

In [0]:
#DA CONTOLLARE CHE ALVUNI  NODI HANNO COMMUNITY NEGATIVA
def find_wars(attackGraph):
    #number max of communities 319
    attacks = np.zeros([319, 319]);
    rouges = np.zeros([319, 319]);
    #print(attacks.shape)

    for e in attackGraph.es():
        #print(e.tuple)
        att = e.tuple[0];
        att = attackGraph.vs[att]['id']
        defe = e.tuple[1];
        defe = attackGraph.vs[defe]['id']
        #print('attaccante: ' + str(att));
        #print('difensore: ' + str(defe));
        att_comm = attackGraph.vs.select(id = str(att))['Community'];
        defe_comm = attackGraph.vs.select(id = str(defe))['Community'];
        if(len(att_comm) > 0 and len(defe_comm) > 0):
            att_comm = att_comm[0];
            defe_comm = defe_comm[0];


            if( att_comm != defe_comm):
                attacks[att_comm, defe_comm] += 1;
            if( att_comm == defe_comm):
                rouges[att_comm, defe_comm] += 1;
             
    print(sum(sum(attacks[:,:])));
    print('max: ')
    print(np.amax(attacks[:,:]))
    print("")
    return attacks, rouges;
#for each attack edge
      #if att['community'] != def['community']
        #war[day, att['community'], def['community']] += 1
    #if war[day, att['community'], def['community']] > #min attack number to call it war
      #print('there was a war between A e B')
      
def find_max_war(attackGraph):
    #number max of communities 319
    attacks = np.zeros([319, 319]);
    

    for e in attackGraph.es():
        #print(e.tuple)
        att = e.tuple[0];
        att = attackGraph.vs[att]['id']
        defe = e.tuple[1];
        defe = attackGraph.vs[defe]['id']
      
        att_comm = attackGraph.vs.select(id = str(att))['Community'];
        defe_comm = attackGraph.vs.select(id = str(defe))['Community'];
        
        if(len(att_comm) > 0 and len(defe_comm) > 0):
            att_comm = att_comm[0];
            defe_comm = defe_comm[0];


            if( att_comm != defe_comm):
                attacks[att_comm, defe_comm] += 1;
           
    print(sum(sum(attacks[:,:])));
    print('max: ')
    print(np.amax(attacks[:,:]))
    for r in range(0, 319):
      for c in range(0,319):
        if attacks[r,c] == np.amax(attacks[:,:]):
            print('max number of fight at day 1 is from: ' + str(r) + ' to: ' + str(c))
            return r, c;


          
def war_change_communities(wars):
    info = np.zeros([30,5])


    dies = np.zeros([30,3])
    i= 0
    j = 0
    for day in range(0,30):
      for r in range(0, wars.shape[0]):
        for c in range(wars.shape[1]):
            if (day < 29 and len(communityAffiliation[day]) >= c):
              if (wars[day, r, c] == 1 and len(communityAffiliation[day+1][c][0]) > 0):

                if(communityAffiliation[day+1][c][1] < 1 or communityAffiliation[day+1][c][0][0] == 'DISSOLVED'):

                   if(communityAffiliation[day][c][1] >= 1 and len(communityAffiliation[day][c][0]) > 0 and communityAffiliation[day][c][0][0] != 'DISSOLVED'):

                    print('community: ' + str(c) + ' has died after the war with comm: ' + str(r) + ' at day: ' + str(day+1) + ' of 30 days');
                    info[i, j] = r;
                    info[i, j+1] = c;
                    info[i, j+2] = day+1;
                    info[i, j+3] = communityAffiliation[day][c][1];
                    info[i, j+4] = 0

                    dies[i,j] = r;
                    dies[i,j+1] = c;
                    dies[i,j+2] = day+1;
                    i += 1;

    print(info)
    maxx = 0;
    max_c = 0;
    for day in range(0,30):
      for comm in dies[:,1]:
        if (len(communityAffiliation[day]) > comm):
          '''print('community ' + str(comm))
          print('day ' + str(day))
          print(len(communityAffiliation[day]))
          print(communityAffiliation[day][int(comm)][1])'''
          if (communityAffiliation[day][int(comm)][1] > maxx):
            maxx = communityAffiliation[day][int(comm)][1]
            max_c = comm;

    print(max_c)
    print(maxx)
    for c in dies[:,1]:
      size = 0;
      for day in range(0,30):
        if (len(communityAffiliation[day]) > c):
          if (len(communityAffiliation[day][c][0]) > 0):
            if (communityAffiliation[day][c][1] < 1 or communityAffiliation[day][c][0][0] == 'DISSOLVED'):
              size = 0;
            else:
              size = communityAffiliation[day][c][1];
          print('Community: ' + str(c) + ' at day: ' + str(day) + ' has #member: '+ str(size) )

    
def war_change_leader(war, leaders, attackGraph):
    print('a')
    #if war[day, A, B]
      #if attackGraph(leaders[day, B]).inDegree() > #N attacchi
        #if leaders[day + 1, B] != leaders[day, B]
          #print('B has a new leader after the war at day: ' +  day)
          
          
def search_for_leaders(comm_dict, allianceGraph, AttackGraph):
  print('a')
  #leaders = dict()
  #for node in alliance
    #if node.centrality is
def max_attacked_community(wars):
  lotte = np.zeros([30, 2])
  for day in range(0,30):
    fight = sum(wars[day,:,:])
    maxi = np.amax(fight)
    print('New max: ' + str(maxi))
    #print(fight)
    for c in range(0,319):
        #print(fight)
        if (fight[c] == maxi):
            print('max number of fight at day: ' + str(day) + ' is ' + str(c))
            lotte[day, 0] = c
            lotte[day, 1] = maxi
  print(lotte)

In [76]:
max_attacked_community(wars)


New max: 73.0
max number of fight at day: 0 is 5
New max: 81.0
max number of fight at day: 1 is 5
New max: 73.0
max number of fight at day: 2 is 5
New max: 65.0
max number of fight at day: 3 is 5
New max: 55.0
max number of fight at day: 4 is 5
New max: 70.0
max number of fight at day: 5 is 5
New max: 65.0
max number of fight at day: 6 is 5
New max: 71.0
max number of fight at day: 7 is 5
New max: 71.0
max number of fight at day: 8 is 5
New max: 68.0
max number of fight at day: 9 is 5
New max: 80.0
max number of fight at day: 10 is 5
New max: 68.0
max number of fight at day: 11 is 5
New max: 62.0
max number of fight at day: 12 is 5
New max: 64.0
max number of fight at day: 13 is 5
New max: 70.0
max number of fight at day: 14 is 5
New max: 66.0
max number of fight at day: 15 is 5
New max: 73.0
max number of fight at day: 16 is 5
New max: 73.0
max number of fight at day: 17 is 5
New max: 75.0
max number of fight at day: 18 is 5
New max: 63.0
max number of fight at day: 19 is 5
New max: 6

Add community label to attackGraph

In [0]:
communityAffiliation = dict()

communityStory = dict()
    
for i in range(0, len(allNetwork)):
    # print(allNetwork[i][4].summary())
    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][0], i)
    # addCommunitiesLabel(allNetwork[i][3], i)
    print(allNetwork[i][4].summary())
print("")

print("Finito")

In [55]:

wars = np.zeros([30, 319, 319]);
rouge = np.zeros([30, 319, 319]);
for i in range(0,30):
    attacks, rouges = find_wars(allNetwork[i][0]);
    wars[i, :, :] = attacks;
    rouge[i, :, :] = rouges;
print('total number of edges: ')
print(sum(sum(wars[0,:,:])) + sum(sum(rouge[0,:,:])));
print(allNetwork[0][0].summary())



r,c = find_max_war(allNetwork[i][0])
war_change_communities(wars);

2091.0
max: 
15.0

1998.0
max: 
23.0

2068.0
max: 
21.0

1786.0
max: 
17.0

1688.0
max: 
19.0

1862.0
max: 
19.0

1893.0
max: 
28.0

1819.0
max: 
23.0

1774.0
max: 
16.0

1769.0
max: 
17.0

1950.0
max: 
23.0

1826.0
max: 
25.0

1833.0
max: 
29.0

1785.0
max: 
24.0

1682.0
max: 
21.0

1722.0
max: 
19.0

1726.0
max: 
20.0

1769.0
max: 
22.0

1577.0
max: 
18.0

1493.0
max: 
19.0

1561.0
max: 
17.0

1570.0
max: 
21.0

1336.0
max: 
17.0

1262.0
max: 
15.0

1282.0
max: 
13.0

1362.0
max: 
14.0

1370.0
max: 
13.0

1347.0
max: 
16.0

1394.0
max: 
15.0

1266.0
max: 
13.0

total number of edges: 
2134.0
IGRAPH D--- 1548 2134 -- 
+ attr: Community (v), id (v), Weight (e)
1266.0
max: 
13.0
max number of fight at day 1 is from: 73 to: 81
community: 14 has died after the war with comm: 9 at day: 3 of 30 days
community: 231 has died after the war with comm: 23 at day: 3 of 30 days
community: 91 has died after the war with comm: 2 at day: 4 of 30 days
community: 91 has died after the war with comm: 8 

In [19]:
war_change_communities(wars);

NameError: ignored

In [31]:
x = np.ones([3,4])
print(sum(x))

[3. 3. 3. 3.]


In [0]:
#att, def, day, from, to
info = np.zeros([30,5])

dies = np.zeros([30,3])
i= 0
j = 0
for day in range(0,30):
  for r in range(0, wars.shape[0]):
    for c in range(wars.shape[1]):
        if (day < 29 and len(communityAffiliation[day]) >= c):
          if (wars[day, r, c] == 1 and len(communityAffiliation[day+1][c][0]) > 0):
            
            if(communityAffiliation[day+1][c][1] < 1 or communityAffiliation[day+1][c][0][0] == 'DISSOLVED'):
             
               if(communityAffiliation[day][c][1] >= 1 and len(communityAffiliation[day][c][0]) > 0 and communityAffiliation[day][c][0][0] != 'DISSOLVED'):
            
                print('community: ' + str(c) + ' has died after the war with comm: ' + str(r) + ' at day: ' + str(day+1) + ' of 30 days');
                info[i, j] = r;
                info[i, j+1] = c;
                info[i, j+2] = day+1;
                info[i, j+3] = communityAffiliation[day][c][1];
                info[i, j+4] = 0
                
                dies[i,j] = r;
                dies[i,j+1] = c;
                dies[i,j+2] = day+1;
                i += 1;
                
print(info)
maxx = 0;
max_c = 0;
for day in range(0,30):
  for comm in dies[:,1]:
    if (len(communityAffiliation[day]) > comm):
      '''print('community ' + str(comm))
      print('day ' + str(day))
      print(len(communityAffiliation[day]))
      print(communityAffiliation[day][int(comm)][1])'''
      if (communityAffiliation[day][int(comm)][1] > maxx):
        maxx = communityAffiliation[day][int(comm)][1]
        max_c = comm;
      
print(max_c)
print(maxx)
for c in dies[:,1]:
  size = 0;
  for day in range(0,30):
    if (len(communityAffiliation[day]) > c):
      if (len(communityAffiliation[day][c][0]) > 0):
        if (communityAffiliation[day][c][1] < 1 or communityAffiliation[day][c][0][0] == 'DISSOLVED'):
          size = 0;
        else:
          size = communityAffiliation[day][c][1];
      print('Community: ' + str(c) + ' at day: ' + str(day) + ' has #member: '+ str(size) )
                
'''else:
            if (wars[day, r, c] == 1 and len(communityAffiliation[day+1][r][0])==0 and len(communityAffiliation[day][r][0])>0):
              print('community: ' + str(r) + ' has died after the war with comm: ' + str(c) + ' at day: ' + str(day));'''
          
'''print(communityAffiliation[0][1][0])
for day in communityAffiliation:
  for comm in communityAffiliation[day]:
    print(len(communityAffiliation[day][comm][0]))'''
      
      
    


community: 14 has died after the war with comm: 9 at day: 3 of 30 days
community: 231 has died after the war with comm: 23 at day: 3 of 30 days
community: 91 has died after the war with comm: 2 at day: 4 of 30 days
community: 91 has died after the war with comm: 8 at day: 4 of 30 days
community: 222 has died after the war with comm: 8 at day: 6 of 30 days
community: 222 has died after the war with comm: 16 at day: 6 of 30 days
community: 222 has died after the war with comm: 23 at day: 6 of 30 days
community: 219 has died after the war with comm: 8 at day: 8 of 30 days
community: 249 has died after the war with comm: 8 at day: 10 of 30 days
community: 249 has died after the war with comm: 16 at day: 10 of 30 days
community: 204 has died after the war with comm: 8 at day: 14 of 30 days
community: 125 has died after the war with comm: 23 at day: 15 of 30 days
community: 160 has died after the war with comm: 23 at day: 17 of 30 days
community: 215 has died after the war with comm: 23 at d

'print(communityAffiliation[0][1][0])\nfor day in communityAffiliation:\n  for comm in communityAffiliation[day]:\n    print(len(communityAffiliation[day][comm][0]))'

In [0]:
for i in range(0, len(allNetwork)):
    print(allNetwork[i][4].summary())
    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][4], i)
    print(allNetwork[i][4].summary())
communitiesStory = dict()

communitiesStory = retrieveCommunitiesGraph(allNetwork, 14, 9, 4)
communitiesStory[1].save(baseURL + "Dataset/day1.graphml", format="graphml")
communitiesStory[2].save(baseURL + "Dataset/day2.graphml", format="graphml")
communitiesStory[3].save(baseURL + "Dataset/day3.graphml", format="graphml")
communitiesStory[4].save(baseURL + "Dataset/day4.graphml", format="graphml")

IGRAPH D--- 3757 18792 -- 
+ attr: id (v), Label (e), Weight (e), key (e)
DAY:  1
NUMBER OF NODES IN DAY TOTAL GRAPH:  3757
NUMBER OF NODES IN COMMUNITIES FILE:  2666
NUMBER OF COMMUNITIES:  232
NUMBER OF DISSOLVED COMMUNITIES:  0
NUMBER OF REAL COMMUNITIES:  232
NUMBER OF NODES IN COMMUNITIES DAY FILE NL PRESENT IN DAY GRAPH:  208
NUMBER OF NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE:  1299
NUMBER OF NODES IN COMMUNITIES DAY FILE PRESENT IN DAY GRAPH:  2458
ALL DAY GRAPH AFTER REMOVING THE NO COMMUNITIES NODE:  IGRAPH D--- 2458 12837 -- 
+ attr: Community (v), id (v), Label (e), Weight (e), key (e)
----------------------------------
IGRAPH D--- 2458 12837 -- 
+ attr: Community (v), id (v), Label (e), Weight (e), key (e)
IGRAPH D--- 3758 17865 -- 
+ attr: id (v), Label (e), Weight (e), key (e)
DAY:  2
NUMBER OF NODES IN DAY TOTAL GRAPH:  3758
NUMBER OF NODES IN COMMUNITIES FILE:  2629
NUMBER OF COMMUNITIES:  235
NUMBER OF DISSOLVED COMMUNITIES:  4
NUMBER OF REAL COMMUNITIES: 

#### Community Centrality

In [0]:
def findDiplomats(allGraph, communityId):
    print("DIPLOMATS DETECTION")
    
    diplomats = dict()
    
    for day in allGraph:
        nToPrint = 5
        print("DAY: ", (day+1))
        edgeBet = allGraph[day][1].edge_betweenness()
        orderEdgeBet = np.argsort(edgeBet)[::-1]
        orderEdgeBet = np.argsort(edgeBet)[::-1]
        print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
          format("From", "To", "From Community", "To Community", "Edge Betweennees", "Weight"))
        for e in orderEdgeBet:
            if allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community'] != allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['Community'] and abs(allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community']) == communityId:
                if nToPrint == 0:
                    break
                print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
                    format(allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id'], allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['id'], 
                             allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['Community'], allGraph[day][1].vs[allGraph[day][1].es[e].tuple[1]]['Community'], 
                             round(edgeBet[e], 5), round(allGraph[day][1].es[e]['Weight']), 0))
                if allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id'] in diplomats:
                    diplomats[allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id']] += 1
                else:
                    diplomats[allGraph[day][1].vs[allGraph[day][1].es[e].tuple[0]]['id']] = 1
                nToPrint -=1
        # print(diplomats)
        print("")

### Visualize Network
TODO

In [0]:
def VisualizeNetwork(allianceNet):
    print("USING Plotly Python Open Source Graphing Library")

# Study about major community in 30 days

In [ ]:
# def communityStudy(allNet, communityId):
#    communityGraph = dict()
#    plotToSave = ig.Graph()
    
    communityGraph = retrieveCommunityGraph(allNet, communityId, 3)
    
    
    
    # for day in communityGraph:
    #    plotToSave = ig.plot(communityGraph[day])
    #    plotToSave.save(baseURL + "Dash/NormalGraph/day"+str(day+1)+".png")
        

## Main
From cell below we let's start all our analysis

In [0]:
#                   TRY, IF YOU WANT!                 #
 ######################################################
#userNumberTrend(allNetwork)                           #
#edgeNumberTrend(allNetwork)                           #
#tricksterAnalysis(allNetwork)                         #
#communityMemberTrend(communityAffiliation, 0)       #
#communitiesNumberTrend(communityAffiliation)          #
#dissolvedCommunitiesTrend(communityAffiliation)       #
#creationCommunitiesTrend(communityAffiliation)        # 
#retrieveCommunityGraph(allNetwork, 99, 4)#
findDiplomats(allNetwork, 5)                          #                     
 ######################################################

# communityStudy(allNetwork, 5)

# communityGraph = retrieveCommunityGraph(allNetwork, 5, 3)    

for day in communityGraph:
    plotToSave = ig.plot(communityGraph[day])
    plotToSave.save(baseURL +'Dash/SingleCommunityStudy/PlotAllDays/day'+str(day+1))

userCommunityNumberTrend(communityGraph)
edgeCommunityNumberTrend(communityGraph)
    
# print(communityGraph[1].summary())   
    
    
    
    
    
    
    
    
    
    
    
    
# communityAffiliation = dict()

# for i in range(0, len(allNetwork)):
#    print(allNetwork[i][4].summary())
#    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][4], i)
#    print(allNetwork[i][4].summary())
# print("")

# print(communityAffiliation[4])

# with open(baseURL+'Dash/CommunitiesActivityTrend.csv', 'w') as csvfile:
#    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
#    filewriter.writerow(['day', 'CommunityId', 'NumberOfMemberInvolved', 'NumberOfMemberShouldBe', 'NumberOfActive'])
    
#    for day in communityAffiliation:
#        for community in communityAffiliation[day]:
#            filewriter.writerow([int(day+1), community, len(communityAffiliation[day][community][0]),
#                                 communityAffiliation[day][community][1], communityAffiliation[day][community][2]])


# communitiesStory = dict()

communitiesStory = retrieveCommunitiesGraph(allNetwork, (8, 5, 69, 45), 4)
# findDiplomats(allNetwork, 0) 


communitiesStory[0].save(baseURL + "Dataset/day0.graphml", format="graphml")
communitiesStory[1].save(baseURL + "Dataset/day1.graphml", format="graphml")
communitiesStory[2].save(baseURL + "Dataset/day2.graphml", format="graphml")
communitiesStory[3].save(baseURL + "Dataset/day3.graphml", format="graphml")

# print(allNetwork[0][0].summary())

# print(allNetwork[0][3].summary())

# print(communityAffiliation[4][140])

DIPLOMATS DETECTION
DAY:  1
 From     To  From Community    To Community  Edge Betweennees         Weight
    2   6371               0             155       40693.75932              1
    2   5684               0              44       34262.89001              1
    2   5841               0               8       17866.87989              1
    2   1808               0            -216       15448.12667              1
    2   7923               0             117       10246.50833              1

DAY:  2
 From     To  From Community    To Community  Edge Betweennees         Weight
    1   3403               0             146        48165.0111              1
    1   4733               0             155       17829.26193              1
    1   6378               0              84       15987.87843              1
    1   2414               0             131       15507.16801              2
    2   6378               0              84        12719.2105              1

DAY:  3
 From     To  From

 From     To  From Community    To Community  Edge Betweennees         Weight
    1   3403               0             146       27894.13974              2
    1   2145               0             190       17900.76079              1
    2   2975               0              24       17360.95174              1
    2   6371               0             256        6448.68061              1
    1   5668               0             146        3686.71991              1

DAY:  21
 From     To  From Community    To Community  Edge Betweennees         Weight
    2   6371               0             256        37747.3532              1
    1   3403               0             146       21066.08116              1
    1   1804               0              73        9099.13158              1
    2     31               0             190        5079.81308              1
    1   2216               0              73        4341.85131              2

DAY:  22
 From     To  From Community    To Community

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete

# Creation of compact graph (attack, trade and message) and Total Graph and the Alliance Graph (compact and not)
You don't have to run this code if you already have all the graph files in your dataset graphml folder; but **run for another one check**


## Create compact attack graph

In [0]:
# Creation of 30 plain graphs
nGraph = 30
compactAttackGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

attackCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    attackGraph = nx.MultiDiGraph()
    compactAttackGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(attackGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(attackGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                attackCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            attackCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactAttackGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(attackGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAttackGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAttackGraph, baseURL + "Dataset/Graphml-File/Compact/AttackGraph/cAttack-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553

FROM VERTICES NUMBER IN DICTIONARY:  938
START GRAPH ALL VERTICES:  2807
VERTICES WITH OUTDEGREE 0: 1869

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2807
Number of edges: 7476
Average in degree:   2.6633
Average out degree:   2.6633

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  32997
NUMBER OF STEP (END GRAPH EDGES NUMBER):  7476
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995

FROM VERTICES NUMBER IN DICTIONARY:  927
START GRAPH ALL VERTICES:  2802
VERTICES WITH OUTDEGREE 0: 1875

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2802
Number of edges: 7177
Average in degree:   2.5614
Average out degree:   2.5614

SUM OF

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1992
Number of edges: 19969
Average in degree:  10.0246
Average out degree:  10.0246

FROM VERTICES NUMBER IN DICTIONARY:  764
START GRAPH ALL VERTICES:  1992
VERTICES WITH OUTDEGREE 0: 1228

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1992
Number of edges: 4587
Average in degree:   2.3027
Average out degree:   2.3027

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  19969
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4587
NUMBER OF ATTACK EDGES MET:  19969
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1990
Number of edges: 22134
Average in degree:  11.1226
Average out degree:  11.1226

FROM VERTICES NUMBER IN DICTIONARY:  742
START GRAPH ALL VERTICES:  1990
VERTICES WITH OUTDEGREE 0: 1248

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1990
Number of edges: 4705
Average in degree:   2.3643
Average out degree:   2.3643

SUM 

DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1685
Number of edges: 16513
Average in degree:   9.8000
Average out degree:   9.8000

FROM VERTICES NUMBER IN DICTIONARY:  646
START GRAPH ALL VERTICES:  1685
VERTICES WITH OUTDEGREE 0: 1039

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1685
Number of edges: 4008
Average in degree:   2.3786
Average out degree:   2.3786

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  16513
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4008
NUMBER OF ATTACK EDGES MET:  16513
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1643
Number of edges: 16749
Average in degree:  10.1942
Average out degree:  10.1942

FROM VERTICES NUMBER IN DICTIONARY:  643
START GRAPH ALL VERTICES:  1643
VERTICES WITH OUTDEGREE 0: 1000

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1643
Number of edges: 3727
Average in degree:   2.2684
Average out degree:   2.2684

SUM 

## Create compact trade graph

In [0]:
# Creation of 30 plain graphs
nGraph = 30
compactTradeGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

cont = 0
tradeCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    tradeGraph = nx.MultiDiGraph()
    compactTradeGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(tradeGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(tradeGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                tradeCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            tradeCount += 1

    
    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if fromVertex == "2789":
                print(fromVertex)
                cont += edgesDict[fromVertex][toVertex]
            nStep += 1
            compactTradeGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(tradeGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactTradeGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    nx.write_graphml(compactTradeGraph, baseURL + "Dataset/Graphml-File/Compact/TradeGraph/cTrade-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')
print(cont)
    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284

2789
FROM VERTICES NUMBER IN DICTIONARY:  1209
START GRAPH ALL VERTICES:  1336
VERTICES WITH OUTDEGREE 0: 127

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1336
Number of edges: 4909
Average in degree:   3.6744
Average out degree:   3.6744

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  9390
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4909
NUMBER OF TRADE EDGES MET:  9390
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1314
Number of edges: 8710
Average in degree:   6.6286
Average out degree:   6.6286

2789
FROM VERTICES NUMBER IN DICTIONARY:  1188
START GRAPH ALL VERTICES:  1314
VERTICES WITH OUTDEGREE 0: 126

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1314
Number of edges: 4659
Average in degree:   3.5457
Average out degree:   3.5457

S

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1199
Number of edges: 10175
Average in degree:   8.4862
Average out degree:   8.4862

2789
FROM VERTICES NUMBER IN DICTIONARY:  1097
START GRAPH ALL VERTICES:  1199
VERTICES WITH OUTDEGREE 0: 102

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1199
Number of edges: 4799
Average in degree:   4.0025
Average out degree:   4.0025

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  10175
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4799
NUMBER OF TRADE EDGES MET:  10175
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1146
Number of edges: 9866
Average in degree:   8.6091
Average out degree:   8.6091

2789
FROM VERTICES NUMBER IN DICTIONARY:  1053
START GRAPH ALL VERTICES:  1146
VERTICES WITH OUTDEGREE 0: 93

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1146
Number of edges: 4924
Average in degree:   4.2967
Average out degree:   4.296

DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 991
Number of edges: 8606
Average in degree:   8.6842
Average out degree:   8.6842

FROM VERTICES NUMBER IN DICTIONARY:  896
START GRAPH ALL VERTICES:  991
VERTICES WITH OUTDEGREE 0: 95

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 991
Number of edges: 4125
Average in degree:   4.1625
Average out degree:   4.1625

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  8606
NUMBER OF STEP (END GRAPH EDGES NUMBER):  4125
NUMBER OF TRADE EDGES MET:  8606
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 967
Number of edges: 8521
Average in degree:   8.8118
Average out degree:   8.8118

FROM VERTICES NUMBER IN DICTIONARY:  880
START GRAPH ALL VERTICES:  967
VERTICES WITH OUTDEGREE 0: 87

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 967
Number of edges: 4004
Average in degree:   4.1406
Average out degree:   4.1406

SUM OF WEIGHT BEFOR

## Create compact message graph

In [0]:
# Creation of 30 plain graphs
nGraph = 30
compactMessageGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

messageCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    messageGraph = nx.MultiDiGraph()
    compactMessageGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(messageGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(messageGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                edgesDict[itr[0]][itr[1]] += 1
                messageCount += 1
            else:
                edgesDict[itr[0]][itr[1]] = 1
                messageCount += 1
        else:
            edgesDict[itr[0]] = {}
            edgesDict[itr[0]][itr[1]] = 1
            messageCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            compactMessageGraph.add_edge(int(fromVertex), int(toVertex), Weight = edgesDict[fromVertex][toVertex])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += edgesDict[fromVertex][toVertex]

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(messageGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactMessageGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(compactMessageGraph, baseURL + "Dataset/Graphml-File/Compact/MessageGraph/cMessage-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271

FROM VERTICES NUMBER IN DICTIONARY:  1168
START GRAPH ALL VERTICES:  1963
VERTICES WITH OUTDEGREE 0: 795

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1963
Number of edges: 6485
Average in degree:   3.3036
Average out degree:   3.3036

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  16346
NUMBER OF STEP (END GRAPH EDGES NUMBER):  6485
NUMBER OF MESSAGE EDGES MET:  16346
----------------------------------------------------------------------
DAY:  2
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2100
Number of edges: 15019
Average in degree:   7.1519
Average out degree:   7.1519

FROM VERTICES NUMBER IN DICTIONARY:  1143
START GRAPH ALL VERTICES:  2100
VERTICES WITH OUTDEGREE 0: 957

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2100
Number of edges: 6105
Average in degree:   2.9071
Average out degree:   2.9071

SUM O

DAY:  15
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1777
Number of edges: 11805
Average in degree:   6.6432
Average out degree:   6.6432

FROM VERTICES NUMBER IN DICTIONARY:  969
START GRAPH ALL VERTICES:  1777
VERTICES WITH OUTDEGREE 0: 808

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1777
Number of edges: 5302
Average in degree:   2.9837
Average out degree:   2.9837

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  11805
NUMBER OF STEP (END GRAPH EDGES NUMBER):  5302
NUMBER OF MESSAGE EDGES MET:  11805
----------------------------------------------------------------------
DAY:  16
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1785
Number of edges: 13474
Average in degree:   7.5485
Average out degree:   7.5485

FROM VERTICES NUMBER IN DICTIONARY:  931
START GRAPH ALL VERTICES:  1785
VERTICES WITH OUTDEGREE 0: 854

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1785
Number of edges: 5700
Average in degree:   3.1933
Average out degree:   3.1933

SUM O

DAY:  29
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1408
Number of edges: 7587
Average in degree:   5.3885
Average out degree:   5.3885

FROM VERTICES NUMBER IN DICTIONARY:  775
START GRAPH ALL VERTICES:  1408
VERTICES WITH OUTDEGREE 0: 633

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1408
Number of edges: 3174
Average in degree:   2.2543
Average out degree:   2.2543

SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  7587
NUMBER OF STEP (END GRAPH EDGES NUMBER):  3174
NUMBER OF MESSAGE EDGES MET:  7587
----------------------------------------------------------------------
DAY:  30
START GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 1512
Number of edges: 8468
Average in degree:   5.6005
Average out degree:   5.6005

FROM VERTICES NUMBER IN DICTIONARY:  734
START GRAPH ALL VERTICES:  1512
VERTICES WITH OUTDEGREE 0: 778

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1512
Number of edges: 3492
Average in degree:   2.3095
Average out degree:   2.3095

SUM OF WE

## Create Total and Alliance Graph

In [0]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

attackSelf = 0
messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    attackSelf = 0
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label = "Trade", time = itr[3]['edgetime'])

    #for itr in attackGraph.edges_iter(None, True, True):
    for itr in list(attackGraph.edges(None, True, True)):
        if int(itr[0]) != int(itr[1]):
            realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
        else:
            attackSelf += 1
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(allianceGraph, baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("NO SENSE SELF ATTACK EDGE: ", attackSelf)
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
NO SENSE SELF MESSAGE EDGE:  515
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
NO SENSE SELF ATTA

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2156
Number of edges: 22162
Average in degree:  10.2792
Average out degree:  10.2792
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1811
Number of edges: 12486
Average in degree:   6.8945
Average out degree:   6.8945
NO SENSE SELF MESSAGE EDGE:  456
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1205
Number of edges: 9008
Average in degree:   7.4755
Average out degree:   7.4755
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2012
Number of edges: 21038
Average in degree:  10.4563
Average out degree:  10.4563
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3011
Number of edges: 43200
Average in degree:  14.3474
Average out degree:  14.3474

Graph number:  12
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
NO SENSE SELF ATTACK EDGE:  0
MESS

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1828
Number of edges: 18771
Average in degree:  10.2686
Average out degree:  10.2686
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1572
Number of edges: 11405
Average in degree:   7.2551
Average out degree:   7.2551
NO SENSE SELF MESSAGE EDGE:  295
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1010
Number of edges: 8443
Average in degree:   8.3594
Average out degree:   8.3594
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1735
Number of edges: 19553
Average in degree:  11.2697
Average out degree:  11.2697
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2658
Number of edges: 38324
Average in degree:  14.4184
Average out degree:  14.4184

Graph number:  22
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1775
Number of edges: 16970
Average in degree:   9.5606
Average out degree:   9.5606
NO SENSE SELF ATTACK EDGE:  0
MESS

## Transformation of the Alliance graph to compact plain graph with weighted edges

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:
#Creation of 30 plain graphs
nGraph = 30
realAllianceGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
cont = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    allianceGraph = nx.MultiDiGraph()
    realAllianceGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Alliance-Network-Graphml/Alliance-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(allianceGraph.edges(None, True, True)):
        if itr[0] == '7855':
            cont += 1
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
            else:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tradeCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), MWeight = edgesDict[fromVertex][toVertex][0], TWeight = edgesDict[fromVertex][toVertex][1])
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allianceGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(realAllianceGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(realAllianceGraph, baseURL + "Dataset/Graphml-File/Compact/AllianceGraph/cAlliance-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)
    

DAY:  1
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711

FROM VERTICES NUMBER IN DICTIONARY:  1707
START GRAPH ALL VERTICES:  2218
VERTICES WITH OUTDEGREE 0: 511

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2218
Number of edges: 11077
Average in degree:   4.9941
Average out degree:   4.9941

NUMBER OF STEP (END GRAPH EDGES NUMBER):  11077
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2311
Number of edges: 23310
Average in degree:  10.0865
Average out degree:  10.0865

FROM VERTICES NUMBER IN DICTIONARY:  1659
START GRAPH ALL VERTICES:  2311
VERTICES WITH OUTDEGREE 0: 652

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 2311
Number of edges: 10448
Average in degree:   4.5210
Average out degree:   4.5210

NUMBER

----------------------------------------------------------------------
7
DAY:  15
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1948
Number of edges: 21580
Average in degree:  11.0780
Average out degree:  11.0780

FROM VERTICES NUMBER IN DICTIONARY:  1439
START GRAPH ALL VERTICES:  1948
VERTICES WITH OUTDEGREE 0: 509

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1948
Number of edges: 9746
Average in degree:   5.0031
Average out degree:   5.0031

NUMBER OF STEP (END GRAPH EDGES NUMBER):  9746
NUMBER OF TRADE EDGES MET:  10175
NUMBER OF MESSAGE EDGES MET:  11405
----------------------------------------------------------------------
8
DAY:  16
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1925
Number of edges: 23046
Average in degree:  11.9719
Average out degree:  11.9719

FROM VERTICES NUMBER IN DICTIONARY:  1385
START GRAPH ALL VERTICES:  1925
VERTICES WITH OUTDEGREE 0: 540

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1925
Number of edg

----------------------------------------------------------------------
22
DAY:  29
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1595
Number of edges: 15994
Average in degree:  10.0276
Average out degree:  10.0276

FROM VERTICES NUMBER IN DICTIONARY:  1192
START GRAPH ALL VERTICES:  1595
VERTICES WITH OUTDEGREE 0: 403

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1595
Number of edges: 7064
Average in degree:   4.4288
Average out degree:   4.4288

NUMBER OF STEP (END GRAPH EDGES NUMBER):  7064
NUMBER OF TRADE EDGES MET:  8606
NUMBER OF MESSAGE EDGES MET:  7388
----------------------------------------------------------------------
22
DAY:  30
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1670
Number of edges: 16735
Average in degree:  10.0210
Average out degree:  10.0210

FROM VERTICES NUMBER IN DICTIONARY:  1155
START GRAPH ALL VERTICES:  1670
VERTICES WITH OUTDEGREE 0: 515

END GRAPH
Name: 
Type: DiGraph
Number of nodes: 1670
Number of edg

## Transformation of the total multigraph to a multigraph with weighted edges for each type of interaction

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:

nGraph = 30
compactAllGraph = nx.MultiDiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
attackCount = 0
finalCheck = 0
cont = 0

#for each day 
for i in range(0, nGraph):
    
    #create new plaingraphs 
    allGraph = nx.MultiDiGraph()
    compactAllGraph = nx.MultiDiGraph()
    checkGraph = ig.Graph()
    allGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Multi/Total-Network-Graphml/Total-2009-12-"+str(i+1)+".graphml")
    
    #initialize counters
    tradeCount = 0
    messageCount = 0
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    finalCheck = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allGraph))
    print("")

    #create a tmp dictionary to count the numbers of messages, trades and attacks between each couple of nodes 
    for itr in list(allGraph.edges(None, True, True)):
        # print(itr)
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                    
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]][2] += 1
                    attackCount += 1
            else: 
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                    attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                tradeCount += 1
            elif itr[3]['Label'] == 'Attack':
                edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                attackCount += 1
    if '7855' in edgesDict:
        cont += edgesDict['7855']['3778'][1]
   #For each edges saved in the dictionary add it to the plain graph with the correct label 
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if edgesDict[fromVertex][toVertex][0] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="m", Label = "Message", Weight = edgesDict[fromVertex][toVertex][0])
                nStep += 1
            if edgesDict[fromVertex][toVertex][1] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="t", Label = "Trade", Weight = edgesDict[fromVertex][toVertex][1])
                nStep += 1
            if edgesDict[fromVertex][toVertex][2] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="a", Label = "Attack", Weight = edgesDict[fromVertex][toVertex][2])
                nStep += 1
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

      
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allGraph))

    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAllGraph))
    print("")
    
    #Crazy things of Villa
    for itr in list(compactAllGraph.edges(None, True, True)):
        finalCheck += itr[3]['Weight']
    print("PARANOIC CHECK: ", finalCheck)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAllGraph, baseURL + "Dataset/Graphml-File/Compact/TotalGraph/cTotal-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    print(cont)

DAY:  1
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

FROM VERTICES NUMBER IN DICTIONARY:  1872
START GRAPH ALL VERTICES:  3757
VERTICES WITH OUTDEGREE 0: 1885

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 18792
Average in degree:   5.0019
Average out degree:   5.0019

PARANOIC CHECK:  58218
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  58218
NUMBER OF STEP (END GRAPH EDGES NUMBER):  18792
NUMBER OF TRADE EDGES MET:  9390
NUMBER OF MESSAGE EDGES MET:  15831
NUMBER OF ATTACK EDGES MET:  32997
----------------------------------------------------------------------
2
DAY:  2
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3758
Number of edges: 56092
Average in degree:  14.9260
Average out degree:  14.9260

FROM VERTICES NUMBER IN DICTIONARY:  1836
START GRAPH ALL VERTICES:  3758
VERTICES WITH OUTDEGREE 0: 1922

END GRAPH


Name: 
Type: MultiDiGraph
Number of nodes: 2965
Number of edges: 15431
Average in degree:   5.2044
Average out degree:   5.2044

PARANOIC CHECK:  44983
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  44983
NUMBER OF STEP (END GRAPH EDGES NUMBER):  15431
NUMBER OF TRADE EDGES MET:  9310
NUMBER OF MESSAGE EDGES MET:  14045
NUMBER OF ATTACK EDGES MET:  21628
----------------------------------------------------------------------
6
DAY:  13
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 44180
Average in degree:  15.0426
Average out degree:  15.0426

FROM VERTICES NUMBER IN DICTIONARY:  1599
START GRAPH ALL VERTICES:  2937
VERTICES WITH OUTDEGREE 0: 1338

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2937
Number of edges: 15199
Average in degree:   5.1750
Average out degree:   5.1750

PARANOIC CHECK:  44180
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  44180
NUMBER OF STEP (END GRAPH EDGES NUMBER):  15199
NUMBER OF TRA

----------------------------------------------------------------------
16
DAY:  24
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2566
Number of edges: 36612
Average in degree:  14.2681
Average out degree:  14.2681

FROM VERTICES NUMBER IN DICTIONARY:  1373
START GRAPH ALL VERTICES:  2566
VERTICES WITH OUTDEGREE 0: 1193

END GRAPH
Name: 
Type: MultiDiGraph
Number of nodes: 2566
Number of edges: 11987
Average in degree:   4.6715
Average out degree:   4.6715

PARANOIC CHECK:  36612
SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER):  36612
NUMBER OF STEP (END GRAPH EDGES NUMBER):  11987
NUMBER OF TRADE EDGES MET:  8659
NUMBER OF MESSAGE EDGES MET:  11486
NUMBER OF ATTACK EDGES MET:  16467
----------------------------------------------------------------------
17
DAY:  25
START GRAPH
Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2525
Number of edges: 31765
Average in degree:  12.5802
Average out degree:  12.5802

FROM VERTICES NUMBER IN DICTIONARY:  130

# Community reorganization in the period

In [0]:
trueTable = dict()
dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-1.txt", "r")
dictDayCommunities = dict()
nNodes = 0
nCommunity = 0
communityMember = list()
communityAnalogy = list()
nSameMember = 0
nGraph = 30
newCommunity = 0

for community in dayCommunities:
    communityMember = list()
    community = community.split()
    for node in community:
        nNodes += 1
        communityMember.append(node)
    trueTable[nCommunity] = list(communityMember)
    nCommunity += 1
    
print("------TRUE TABLE-------")
print("{:<39}{:<4}".format("TRUE TABLE IS COMPOSED BY NODES: ", nNodes))
print("{:<39}{:<4}".format("TRUE TABLE IS COMPOSED BY COMMUNUTIES: ", nCommunity))
print("")

for i in range(2, nGraph+1):
    dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-"+str(i)+".txt", "r")
    nNodes = 0
    nCommunity = 0
    for community in dayCommunities:
        nCommunity += 1
        community = community.split()
        for node in community:
            nNodes += 1
    print("DAY: ", i)
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY NODES: ", nNodes))
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY COMMUNUTIES: ", nCommunity))
    print("")


for i in range(1, nGraph + 1):
    dayCommunities = open(baseURL +  "Dataset/Communities/StartCommunities/communities-2009-12-"+str(i)+".txt", "r")
    dayOrganizeCommunities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(i)+".txt","w+")
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    communityMember = list()
    row = 0
    
    if i > 2:
        trueTable = dict()
        trueCommunities = open(baseURL +  "Dataset/Communities/OrganizedCommunities/Ocommunities-2009-12-"+str(i - 1)+".txt", "r")
        for community in trueCommunities:
            communityMember = list()
            community = community.split()
            for node in community:
                nNodes += 1
                communityMember.append(node)
            trueTable[nCommunity] = list(communityMember)
            nCommunity += 1
        
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    communityMember = list()
    dictDayCommunities = dict()
    row = 0
    
    for community in dayCommunities:
        communityMember = list()
        community = community.split()
        for node in community:
            nNodes += 1
            communityMember.append(node)
        dictDayCommunities[nCommunity] = list(communityMember)
        nCommunity += 1
    
    print("DAY: ", i)
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY NODES: ", nNodes))
    print("{:<39}{:<4}".format("COMMUN DAY IS COMPOSED BY COMMUNUTIES: ", nCommunity + 1))
    print("")
    
    nNodes = 0
    nCommunity = 0
    nSameMember = 0
    row = 0
    communityMember = [0]*(len(trueTable))
    condition = False
    communityChange = list()
    communityStay = list()
    dissolvedCount = 0
    
    # print(trueTable[2])
    # print(dictDayCommunities[2])
    
    for trueCommunity in trueTable:
        communityMember = [0]*len(trueTable)
        for community in dictDayCommunities:
            for vertexA in trueTable[trueCommunity]:
                for vertexB in dictDayCommunities[community]:
                    if vertexB == "DISSOLVED":
                        break
                    if vertexA == vertexB:
                        nSameMember += 1
                        if i == 5:
                            print("{:<16}{:<3}".format("TRUE COMMUNITY: ", trueCommunity))
                            print("{:<16}{:<3}".format("COMMUNITY: ", community))
                            print("VERTEX A: ", vertexA)
                            print("VERTEX B: ", vertexB)
                        if int(trueCommunity) != int(community + dissolvedCount):
                            communityChange.append(int(community + dissolvedCount))
                        else:
                            communityStay.append(int(community + dissolvedCount))
                                            
            if condition:
                print("-----------"+str(trueCommunity)+" VS "+str(community)+"--------------")
                print("SAME MEMBER: ", nSameMember)
                print("TOTAL MEMBER IN TRUE: ", len(trueTable[trueCommunity]))
                print("TOTAL MEMBER IN NEW DAY: ", len(dictDayCommunities[community]))
                print("TRADE: ", nSameMember/len(trueTable[trueCommunity]))
                
            communityMember[community] = nSameMember/len(trueTable[trueCommunity])
            nSameMember = 0
            condition = False
            
        players = ""
        row = communityMember.index(max(communityMember))
        # row += dissolvedCount
        if i == 5:
            print("COMMUNITY INDEX: ", row)
            print(communityMember)
            print("PRESENT IN THIS COMMUNITY", communityChange)
            print("COMMUNITY STAY: ", communityStay)
            print("MELTED COUNT: ", dissolvedCount)
            print("")

        if trueCommunity not in communityStay:
            players += "DISSOLVED"
            dissolvedCount += 1
            if i == 5:
                print("TRUE COMMUNITY: ", trueCommunity)
                print("NOT PRESENTE IN: ", communityStay)
                print("MELTED COUNT: ", dissolvedCount)
        else:
            if row not in communityStay and len(communityStay) > 0:
                print("ROW: ", row)
                print("Community Stay: ", communityStay)
                row = communityStay[0] - dissolvedCount
                print("REAL ROW: ", row)
            for member in dictDayCommunities[row]:
                if '230' in communityStay:
                    print("HERE MAN")
                    print(dictDayCommunities[row])
                players += str(member)
                players += " "
            dictDayCommunities[row].insert(0, "DONE")
        dayOrganizeCommunities.write(players + "\n")
        communityChange.clear()
        communityStay.clear()
        if i == 5:
            print("")
    for community in dictDayCommunities:
        players = ""
        if "DONE" not in dictDayCommunities[community]:
            for member in dictDayCommunities[community]:
                players += str(member)
                players += " "
            dayOrganizeCommunities.write(players + "\n")
    dayOrganizeCommunities.close()
    
    for community in dictDayCommunities:
        if "DONE" not in dictDayCommunities[community]:
            print("New Communities: ", dictDayCommunities[community])
            newCommunity += 1

    
print("NEW COMMUNITY ADDED: ", newCommunity)
        
    
    
                    

    

KeyboardInterrupt: ignored